In [1]:
import glob
import pandas as pd
catfn = glob.glob("train/cat.*.jpg")
dogfn = glob.glob("train/dog.*.jpg")
catans = [0] * len(catfn)
dogans = [1] * len(dogfn)
df = pd.DataFrame({
    "path":catfn + dogfn,
    "ans":catans + dogans
})
df

,ans,path
0,0,train/cat.5077.jpg
1,0,train/cat.2718.jpg
2,0,train/cat.10151.jpg
3,0,train/cat.3406.jpg
4,0,train/cat.4369.jpg
5,0,train/cat.7660.jpg
6,0,train/cat.8553.jpg
7,0,train/cat.9895.jpg
8,0,train/cat.1211.jpg
9,0,train/cat.6218.jpg


In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
from keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [4]:
vgg.layers

In [5]:
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
# trainable一定要在compile前設置
for l in vgg.layers:
    l.trainable = False
x = Flatten()(vgg.output)
x = Dense(512, activation="relu")(x)
x = Dropout(0.25)(x)
x = Dense(2, activation="softmax")(x)
cnn = Model(inputs=vgg.input, outputs=x)
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
cnn.compile(loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"])

In [7]:
import numpy as np
# 順便偷 preprocess_input
from keras.applications.vgg16 import preprocess_input
# Pillow(PIL)
# !pip install Pillow
from keras.preprocessing.image import load_img
img = load_img("train/cat.10000.jpg", target_size=(224, 224))
img_np = np.array(img).astype("float64")
img_pre = preprocess_input(img_np)
img_pre

array([[[ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        ...,
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ]],

       [[ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        ...,
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ]],

       [[ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        ...,
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ],
        [ -95.939, -108.779, -115.68 ]],

       ...,

       [[ -96.939, -101.779,  -41.68 ],
        [ -85.939,  -97.779,  -36.68 ],
        [ -81.939, -101.779,  -41.68 ],
        ...,
        [ -93.939, -108.779,  -96.68 ],
        [ -93.939, -108.779,  -95.68 ],
        [

In [15]:
# return_value: (原圖list, 訓練imglist, 答案list)
def get_imgs(x, y, batch):
    x = np.array(x)
    y = np.array(y)
    # random.randint(下限, 上限)
    # np.random.randint(下限, 上限+1, 你要多少個)
    idx = np.random.randint(0, len(x), batch)
    path = x[idx]

    ori_imgs = []
    pre_imgs = []
    for p in path:
        img = load_img(p, target_size=(224, 224))
        img_np = np.array(img).astype("float64")
        img_pre = preprocess_input(img_np)
        ori_imgs.append(img_np)
        pre_imgs.append(img_pre)
    targets = y[idx]
    return (np.array(ori_imgs), np.array(pre_imgs), np.array(targets))
get_imgs(df["path"], df["ans"], 10)

(array([[[[ -71.68 ,  -65.779,  -73.939],
          [ -71.68 ,  -69.779,  -78.939],
          [ -73.68 ,  -72.779,  -83.939],
          ...,
          [ -29.68 ,    9.221,   35.061],
          [ -30.68 ,    8.221,   34.061],
          [ -33.68 ,    9.221,   36.061]],
 
         [[ -63.68 ,  -57.779,  -65.939],
          [ -58.68 ,  -56.779,  -65.939],
          [ -61.68 ,  -60.779,  -71.939],
          ...,
          [ -26.68 ,   12.221,   38.061],
          [ -27.68 ,   11.221,   37.061],
          [ -30.68 ,   12.221,   39.061]],
 
         [[ -56.68 ,  -51.779,  -59.939],
          [ -55.68 ,  -54.779,  -63.939],
          [ -59.68 ,  -60.779,  -70.939],
          ...,
          [ -26.68 ,   12.221,   38.061],
          [ -27.68 ,   11.221,   37.061],
          [ -31.68 ,   11.221,   38.061]],
 
         ...,
 
         [[ -73.68 ,  -98.779,  -84.939],
          [ -76.68 ,  -99.779,  -86.939],
          [ -78.68 ,  -96.779,  -87.939],
          ...,
          [ -43.68 ,  -35.779,  -

In [17]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
x_train, x_test, y_train, y_test = train_test_split(np.array(df["path"]),
                                                    np.array(df["ans"]),
                                                    test_size=0.2)
# 很衰的時候, 有可能y_test完全都是0, 就會轉錯, 帶入第二個參數
y_train_cat = to_categorical(y_train, 2)
y_test_cat = to_categorical(y_test, 2)
print(y_test[0], y_test_cat[0])

1 [0. 1.]


In [ ]:
# train_on_batch: 自訂訓練流程
batch = 10
for i in range(10):
    _, x, y = get_imgs(x_train, y_train_cat, batch)
    loss = cnn.train_on_batch(x, y)
    print(i, "[Train Loss]:", loss)
    _, x, y = get_imgs(x_test, y_test_cat, batch)
    loss = cnn.test_on_batch(x, y)
    print(i, "[Test Loss]:", loss)

0 [Train Loss]: [3.223619, 0.8]
0 [Test Loss]: [1.6118095, 0.9]
1 [Train Loss]: [9.670857, 0.4]
1 [Test Loss]: [8.059048, 0.5]
2 [Train Loss]: [4.8354287, 0.7]
2 [Test Loss]: [6.447238, 0.6]
3 [Train Loss]: [9.670857, 0.4]
3 [Test Loss]: [11.282667, 0.3]
